In [30]:
import numpy as np
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline

# Inferring neonic toxicity from feeding study data and the VarroaPop+Pesticide model

#### 5/3/19 update

For this update, I looked at our inferred model's predictions to determine the no-observed-adverse-effect concentration (NOAEC) and the lowest-observed-adverse-effect concentration (LOAEC) for each endpoint.

We then compare our inferred NOAEC and LOAEC predictions to what was determined from the empirical data.

#### Defining our analysis and plotting functions first

In [31]:
def convert_data_dict(data_dict):
    """
    Convert the dictionary of study populations by trt and time
    into a 3d array (treatment x time x response var). Also add
    a mean for all individuals
    """
    out = {}
    for stat in ["mean", "sd"]:
        out_stat = np.empty([6, 4, 5])
        for h, h_str in enumerate(["Adults", "Pupae", "Larvae", "Eggs", "All"]): #add in "All" here once it's in field data
            for i, i_str in enumerate(["0", "10", "20", "40", "80", "160"]):
                for j, j_str in enumerate(["4", "5", "6", "7"]):
                    if h_str == "All":
                        if stat == "sd":
                            continue
                        else:
                            key_a = i_str+"_"+j_str+"_{}_{}".format("Adults", stat)
                            key_p = i_str+"_"+j_str+"_{}_{}".format("Pupae", stat)
                            key_l = i_str+"_"+j_str+"_{}_{}".format("Larvae", stat)
                            key_e = i_str+"_"+j_str+"_{}_{}".format("Eggs", stat)
                            out_stat[i,j,h] = data_dict[key_a] + data_dict[key_p] + data_dict[key_l] + data_dict[key_e]
                    else:
                        key = i_str+"_"+j_str+"_{}_{}".format(h_str, stat)
                        out_stat[i,j,h] = data_dict[key]
        out[stat] = out_stat
    return out

In [32]:
# Read the full data set for plots/comparison with model output
study_data_all_path = os.path.abspath('data/data_all.csv')
reader = csv.reader(open(study_data_all_path, 'r'))
study_data_all = {}
for row in reader:
   k, v = row
   study_data_all[k] = float(v)

predictions = np.load("data/predictions.npy")
field_data = convert_data_dict(study_data_all)


In [33]:
def effect_period(response, df_in, field_data, pi_level=95):
    df = df_in.copy()  # trt x days x sample x response
    resp_index = ["Adults", "Pupae", "Larvae", "Eggs", "All"].index(response)
    contrasts = np.subtract(df,df[0,:,:,:])
    contrasts = np.delete(contrasts,0,0) #delete our control row
    upper = np.percentile(contrasts, 100-(100-pi_level)/2, axis=2)
    upper = upper[:,6:,:]# remove 6 days before trts started
    effects = [None, None ,None, None, None]
    for i, trt in enumerate(["10", "20", "40", "80", "160"]):
        sig_diff = np.where(upper[i,:, resp_index] < 0,1,0)
        percent_diff = sig_diff.sum() / len(sig_diff) * 100
        effects[i] = (trt,  f'{percent_diff:.{1}f} %')
    return effects

#### Method
To determine significant reductions in our endpoints, I calculated our models predicted difference between each concentration and the control. 

To make these we are, as usual, considering our model's parameters as probability distributions (which we previously inferred) and sampling many times from these distributions to arrive at a distribution of predictions. 

We can define a significant reduction by seeing where our upper 95% prediction inverval drops below 0. I've also quantified what % of time each treatment spends with a significant reduction in the endpoint. 

### Adults

![alt text](figures/posterior_predictions/bee_population/run_3_11/3_11_2_g13/Adults_contrast_field.png "Effect on adults")

In [34]:
effect_period("Adults", predictions, field_data, pi_level=95)

[('10', '1.7 %'),
 ('20', '0.0 %'),
 ('40', '0.0 %'),
 ('80', '47.0 %'),
 ('160', '91.5 %')]

#### Conclusions - Adults
* We infer a NOEAC/NOEAC of 40 and 80 ppb, respectively
* In the feeding study report, they found 20 and 40 ppb

### Pupae

![alt text](figures/posterior_predictions/bee_population/run_3_11/3_11_2_g13/Pupae_contrast_field.png "Effect on pupae")

In [35]:
effect_period("Pupae", predictions, field_data, pi_level=95)

[('10', '0.0 %'),
 ('20', '0.0 %'),
 ('40', '0.0 %'),
 ('80', '47.0 %'),
 ('160', '88.9 %')]

#### Conclusions - Pupae
* We infer a NOEAC/NOEAC of 40 and 80 ppb, respectively
* In the feeding study report, they found 20 and 40 ppb

### Larvae

![alt text](figures/posterior_predictions/bee_population/run_3_11/3_11_2_g13/Larvae_contrast_field.png "Effect on larvae")

In [36]:
effect_period("Larvae", predictions, field_data, pi_level=95)

[('10', '0.0 %'),
 ('20', '0.0 %'),
 ('40', '0.0 %'),
 ('80', '46.2 %'),
 ('160', '87.2 %')]

#### Conclusions - Larvae
* We infer a NOEAC/NOEAC of 40 and 80 ppb, respectively
* In the feeding study report, they found 40 and 80 ppb

### Eggs

![alt text](figures/posterior_predictions/bee_population/run_3_11/3_11_2_g13/Eggs_contrast_field.png "Effect on eggs")

In [37]:
effect_period("Eggs", predictions, field_data, pi_level=95)

[('10', '0.0 %'),
 ('20', '0.0 %'),
 ('40', '0.0 %'),
 ('80', '47.0 %'),
 ('160', '87.2 %')]

#### Conclusions - Eggs
* We infer a NOEAC/NOEAC of 40 and 80 ppb, respectively
* In the feeding study report, they found 40 and 80 ppb

### All individuals

![alt text](figures/posterior_predictions/bee_population/run_3_11/3_11_2_g13/All_contrast_field.png "Effect on all individuals")

In [38]:
effect_period("All", predictions, field_data, pi_level=95)

[('10', '0.0 %'),
 ('20', '0.0 %'),
 ('40', '0.0 %'),
 ('80', '45.3 %'),
 ('160', '99.1 %')]

#### Conclusions - All individuals
* We infer a NOEAC/NOEAC of 40 and 80 ppb, respectively
* In the feeding study report, they found 20 and 40 ppb

### Overall conclusions

Our model consistently predicts a NOAEC/LOAEC of 40/80 ppb clothianidin. 

Statistical tests on the empirical feeding study data found the same NOAEC/LOAEC for eggs and larave, but a lower NOAEC/LOAEC of 20/40 ppb for the adults, pupae, and all individual endpoints. 

The difference for some endpoints reflects how our inferred VarroaPop based model does not predict the same declines at 40 ppb that we see in the empirical data. This likely reflects tradeoffs in fitting a single model to all of these treatment levels and endpoints, and the inconsistently of the 40 ppb effect in the data. If toxicity increases in the model to produce effects at 40 ppb, it likely overpredicts effects at higher concentrations, and in larvae + egg endpoints for 40 ppb.

### To do:

* I am currently working on running predictions at 50, 55, 60, 65 and 70 ppb concentrations, to see if we can predict a more exact NOAEC and LOAEC
* We can estimate acute and chronic levels of concern from our inferred ld50 distribution and our LOAEC